In [1]:
import numpy as np

In [32]:
class Node:
    def __init__(self, parent, state, cost):
        
        self.parent = parent
        self.state = state
        self.cost = cost
    
    def __hash__(self):
        
        return hash(''.join(self.state))
    
    def __str__(self):
        return str(self.state)
    
    def __eq__(self, other):
        
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten())) 
    
    def __ne__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

In [33]:
class PriorityQueue():
    
    def __init__(self):
        self.queue = []
        
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        
        next_state = None
        state_cost = 10**18
        index = -1
        
        for i in range(len(self.queue)):
            
            if self.queue[i].cost<state_cost:
                state_cost = self.queue[i].cost
                index = i
        
        return self.queue.pop(index)
    
    def is_empty(self):
        
        return len(self.queue)==0
    
    def __str__(self):
        l = []
        for i in self.queue:
            l.append(i.state)
        
        return str(l)
    
    def __len__(self):
        return len(self.queue)
            

In [34]:
class Environment():
    
    #index 0 is missionary
    #index 1 is cannibal
    #index 2 indicates boat
    
    def __init__(self, number_rabbits):
        self.actions = [1,2,3,4] #1 - L to R, 2 R to L, 3 L hops, R hops
        self.start_state = ['L']*number_rabbits + ['_'] + ['R']*number_rabbits
        self.goal_state = ['R']*number_rabbits + ['_'] + ['L']*number_rabbits
        self.number_rabbits = number_rabbits
    
    def get_start_state(self):
        return self.start_state
    
    def get_next_states(self, state):
        
        space = 0
        for i in range(self.number_rabbits*2+1):
            if state[i]=='_':
                space = i
                break
        new_states = []
        
        #action 1
        if space != self.number_rabbits*2:
            if state[space+1]=='R':
                new_state = state.copy()
                new_state[space] = 'R'
                new_state[space+1] = '_'
                new_states.append(new_state)
        
        #action 2
        if space != 0:
            if state[space-1]=='L':
                new_state = state.copy()
                new_state[space] = 'L'
                new_state[space-1] = '_'
                new_states.append(new_state)
        
        #action 3
        if space-2>=0:
            if state[space-2]=='L':
                new_state = state.copy()
                new_state[space] = 'L'
                new_state[space-2] = '_'
                new_states.append(new_state)
        
        if space+2<=self.number_rabbits*2:
            if state[space+2]=='R':
                new_state = state.copy()
                new_state[space] = 'R'
                new_state[space+2] = '_'
                new_states.append(new_state)
        
        return new_states
    
    def reached_goal(self, state):
        
        for i in range(self.number_rabbits*2+1):
                if state[i] != self.goal_state[i]:
                    return False
        
        return True

In [43]:
env = Environment(number_rabbits=4)
start_state = env.get_start_state()
explored = dict()
frontier = PriorityQueue()

init_state = env.get_start_state()
init_node = Node(parent = None, state = init_state, cost = 0)
frontier.push(init_node)

In [44]:
goal_node = None
while not frontier.is_empty():
    
    curr_node = frontier.pop()
    next_states = env.get_next_states(curr_node.state)
    
    if hash(curr_node) in explored:
        continue
        
    explored[hash(curr_node)] = curr_node
    
    if env.reached_goal(curr_node.state):
        goal_node = curr_node
        break
    
    for state in next_states:
        node = Node(parent=curr_node, state=state, cost=curr_node.cost+1)
        frontier.push(node)

    

In [45]:
node = goal_node
l = []
while node is not None:
    l.append(node)
    node = node.parent

step = 1
for node in l[::-1]:
    print("Step: ",step)
    print(node)
    print()
    step+=1
    

Step:  1
['L', 'L', 'L', 'L', '_', 'R', 'R', 'R', 'R']

Step:  2
['L', 'L', 'L', 'L', 'R', '_', 'R', 'R', 'R']

Step:  3
['L', 'L', 'L', '_', 'R', 'L', 'R', 'R', 'R']

Step:  4
['L', 'L', '_', 'L', 'R', 'L', 'R', 'R', 'R']

Step:  5
['L', 'L', 'R', 'L', '_', 'L', 'R', 'R', 'R']

Step:  6
['L', 'L', 'R', 'L', 'R', 'L', '_', 'R', 'R']

Step:  7
['L', 'L', 'R', 'L', 'R', 'L', 'R', '_', 'R']

Step:  8
['L', 'L', 'R', 'L', 'R', '_', 'R', 'L', 'R']

Step:  9
['L', 'L', 'R', '_', 'R', 'L', 'R', 'L', 'R']

Step:  10
['L', '_', 'R', 'L', 'R', 'L', 'R', 'L', 'R']

Step:  11
['_', 'L', 'R', 'L', 'R', 'L', 'R', 'L', 'R']

Step:  12
['R', 'L', '_', 'L', 'R', 'L', 'R', 'L', 'R']

Step:  13
['R', 'L', 'R', 'L', '_', 'L', 'R', 'L', 'R']

Step:  14
['R', 'L', 'R', 'L', 'R', 'L', '_', 'L', 'R']

Step:  15
['R', 'L', 'R', 'L', 'R', 'L', 'R', 'L', '_']

Step:  16
['R', 'L', 'R', 'L', 'R', 'L', 'R', '_', 'L']

Step:  17
['R', 'L', 'R', 'L', 'R', '_', 'R', 'L', 'L']

Step:  18
['R', 'L', 'R', '_', 'R', 'L',